# Reward Model Training

This tutorial trains a reward model on "chosen" and "rejected" responses. 

Training Procedure
1. Read a dataset with three columns - instruction, chosen_response and rejected_response.
2. Use HF's `AutoModelForSequenceClassification` to wrap Roberta model to output a binary class probability. 
3. Prepare dataset for Reward Trainer with each data point containing `input_token_ids_chosen` and `input_token_ids_rejected`. 
4. Pass model, dataset, and `RewardConfig` to TRL's `RewardTrainer` and call `train()`.

In [1]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments
from trl import RewardTrainer, RewardConfig

# Read Dataset

Dataset format: (prompt, chosen response, rejected response)

In [ ]:
# Read dataset
df = pd.read_csv('comparisons.csv', dtype={'question':str, 'chosen':str, 'rejected':str})
print("len(df)", len(df))

df.rename(columns={'question':'instruction'}, inplace=True)
df.rename(columns={'chosen':'chosen_response'}, inplace=True)
df.rename(columns={'rejected':'rejected_response'}, inplace=True)

# convert df to dict
rows = df.to_dict(orient='records')
cols = df.to_dict(orient='list')

print("len rows", len(rows))
print("rows[:3]", rows[:3])

print("cols type", type(cols))
print("cols keys", cols.keys())

print("cols['instruction'][:3]", cols['instruction'][:3])

prepared_dataset = Dataset.from_dict(cols)
print(prepared_dataset.to_pandas())

len(df) 18007
len rows 6987
rows[:3] [{'instruction': 'Voiced by Harry Shearer, what Simpsons character was modeled after Ted Koppel?', 'chosen_response': "Apu Nahasapeemapetilon is a recurring character in the American animated television series The Simpsons. He is an Indian immigrant proprietor who runs the Kwik-E-Mart, a popular convenience store in Springfield. [1] He was based on Peter Seller's character in the film The Party. [2]", 'rejected_response': 'The Simpsons character that was possibly based on Ted Koppel is Kent Brockman.  He is a local news anchor in Springfield and is modeled after Ted Koppel. [1]'}, {'instruction': 'Alliumphobia is the irrational fear of which plant?', 'chosen_response': 'Alliumphobia is the irrational fear of garlic. [1]', 'rejected_response': 'Alliumphobia is the fear of garlic [1, 3]. People who suffer from this condition can expect to experience a very high amount of anxiety from merely thinking of garlic, let alone actually seeing it in real life

In [3]:
#Select a base model whch we need to train for reward modeling.
model_name = "distilroberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}
    prompt_plus_chosen_response = examples["instruction"] + "\n" + examples["chosen_response"]
    prompt_plus_rejected_response = examples["instruction"] + "\n" + examples["rejected_response"]
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

formatted_dataset = prepared_dataset.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

training_args = RewardConfig(
    output_dir="./reward_model",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs = 10,
    report_to=None,
)
# Loading the RewardTrainer from TRL
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],
)
trainer.train()

Map: 100%|██████████| 6987/6987 [00:31<00:00, 220.50 examples/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sadmankiba (sadman-ai). Use `wandb login --relogin` to force relogin


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
1,0.726800,0.693517,0.497424
2,0.681000,0.693581,0.502576
3,0.711900,0.693462,0.497424
4,0.703400,0.693256,0.499714
5,0.694500,0.693095,0.497424
6,0.687900,0.692943,0.518031
7,0.680100,0.692809,0.522038
8,0.697600,0.692696,0.521465
9,0.704100,0.692581,0.528334
10,0.707400,0.692514,0.527189


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>why does moving an object, that's covered  │ <s>why does moving an object, that's covered │ [0.575, 0.425]   │
│ in a liquid, up make the liquid drip faster?  │ in a liquid, up make the liquid drip faster? │                  │
│                                               │                                              │                  │
│ My Chinese chicken does this when dipped in   │ My Chinese chicken does this when dipped in  │                  │
│ honey sauce. When I move it up to the tray    │ honey sauce. When I move it up to the tray   │                  │
│ from the sauce which is lower it drips really │ from the sauce which is lower it drips       │                  │
│ fast.                                         │ really fast.                                 │                  │
│ When an object covered in a liquid is moved   │ Gas rises because it is less dense than the  │                  │
│ up, it is causing the temperature of the      │ liquids or solids that surround it.          │                  │
│ liquid to drop [1]. When the temperature of a │ Similarly, waves form in a fluid when the    │                  │
│ liquid drops, it causes condensation, which   │ fluid is repeatedly pulled down by gravity,  │                  │
│ is the transfer of heat from the air into the │ causing the speed of wave formation to       │                  │
│ liquid [1,3]. The cooling process is hastened │ exceed the speed at which the waves          │                  │
│ by the formation of water droplets on the     │ themselves grow. As a result, a fluid can    │                  │
│ object, which suck heat from the air and      │ form strands before the waves grow to the    │                  │
│ transfer it directly into the cold liquid     │ length at which they break apart the         │                  │
│ [1].</s>                                      │ strands. [1, 2] This effect can also be seen │                  │
│                                               │ when honey is dripped from a spoon: the      │                  │
│                                               │ viscosity of honey is not a factor in        │                  │
│                                               │ forming strands since gravity pulls the      │                  │
│                                               │ honey into a long string regardless of       │                  │
│                                               │ viscosity. [1]</s>                           │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ <s>Why didn't bad eyesight get naturally      │ <s>Why didn't bad eyesight get naturally     │ [0.4776, 0.5224] │
│ selected out?                                 │ selected out?                                │                  │
│                                               │                                              │                  │
│ A lot of people have awful eyesight that      │ A lot of people have awful eyesight that     │                  │
│ would make them almost useless in nature, and │ would make them almost useless in nature,    │                  │
│ it's a high enough percentage of the          │ and it's a high enough percentage of the     │                  │
│ population such that I don't think its        │ population such that I don't think its       │                  │
│ survival can be attributed to the same        │ survival can be attributed to the same       │                  │
│ reasons as, say, blindness being present. Did │ reason

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>why does moving an object, that's covered  │ <s>why does moving an object, that's covered │ [0.4886, 0.5114] │
│ in a liquid, up make the liquid drip faster?  │ in a liquid, up make the liquid drip faster? │                  │
│                                               │                                              │                  │
│ My Chinese chicken does this when dipped in   │ My Chinese chicken does this when dipped in  │                  │
│ honey sauce. When I move it up to the tray    │ honey sauce. When I move it up to the tray   │                  │
│ from the sauce which is lower it drips really │ from the sauce which is lower it drips       │                  │
│ fast.                                         │ really fast.                                 │                  │
│ When an object covered in a liquid is moved   │ Gas rises because it is less dense than the  │                  │
│ up, it is causing the temperature of the      │ liquids or solids that surround it.          │                  │
│ liquid to drop [1]. When the temperature of a │ Similarly, waves form in a fluid when the    │                  │
│ liquid drops, it causes condensation, which   │ fluid is repeatedly pulled down by gravity,  │                  │
│ is the transfer of heat from the air into the │ causing the speed of wave formation to       │                  │
│ liquid [1,3]. The cooling process is hastened │ exceed the speed at which the waves          │                  │
│ by the formation of water droplets on the     │ themselves grow. As a result, a fluid can    │                  │
│ object, which suck heat from the air and      │ form strands before the waves grow to the    │                  │
│ transfer it directly into the cold liquid     │ length at which they break apart the         │                  │
│ [1].</s>                                      │ strands. [1, 2] This effect can also be seen │                  │
│                                               │ when honey is dripped from a spoon: the      │                  │
│                                               │ viscosity of honey is not a factor in        │                  │
│                                               │ forming strands since gravity pulls the      │                  │
│                                               │ honey into a long string regardless of       │                  │
│                                               │ viscosity. [1]</s>                           │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ <s>Why didn't bad eyesight get naturally      │ <s>Why didn't bad eyesight get naturally     │ [0.4708, 0.5292] │
│ selected out?                                 │ selected out?                                │                  │
│                                               │                                              │                  │
│ A lot of people have awful eyesight that      │ A lot of people have awful eyesight that     │                  │
│ would make them almost useless in nature, and │ would make them almost useless in nature,    │                  │
│ it's a high enough percentage of the          │ and it's a high enough percentage of the     │                  │
│ population such that I don't think its        │ population such that I don't think its       │                  │
│ survival can be attributed to the same        │ survival can be attributed to the same       │                  │
│ reasons as, say, blindness being present. Did │ reason

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>why does moving an object, that's covered  │ <s>why does moving an object, that's covered │ [0.478, 0.522]   │
│ in a liquid, up make the liquid drip faster?  │ in a liquid, up make the liquid drip faster? │                  │
│                                               │                                              │                  │
│ My Chinese chicken does this when dipped in   │ My Chinese chicken does this when dipped in  │                  │
│ honey sauce. When I move it up to the tray    │ honey sauce. When I move it up to the tray   │                  │
│ from the sauce which is lower it drips really │ from the sauce which is lower it drips       │                  │
│ fast.                                         │ really fast.                                 │                  │
│ When an object covered in a liquid is moved   │ Gas rises because it is less dense than the  │                  │
│ up, it is causing the temperature of the      │ liquids or solids that surround it.          │                  │
│ liquid to drop [1]. When the temperature of a │ Similarly, waves form in a fluid when the    │                  │
│ liquid drops, it causes condensation, which   │ fluid is repeatedly pulled down by gravity,  │                  │
│ is the transfer of heat from the air into the │ causing the speed of wave formation to       │                  │
│ liquid [1,3]. The cooling process is hastened │ exceed the speed at which the waves          │                  │
│ by the formation of water droplets on the     │ themselves grow. As a result, a fluid can    │                  │
│ object, which suck heat from the air and      │ form strands before the waves grow to the    │                  │
│ transfer it directly into the cold liquid     │ length at which they break apart the         │                  │
│ [1].</s>                                      │ strands. [1, 2] This effect can also be seen │                  │
│                                               │ when honey is dripped from a spoon: the      │                  │
│                                               │ viscosity of honey is not a factor in        │                  │
│                                               │ forming strands since gravity pulls the      │                  │
│                                               │ honey into a long string regardless of       │                  │
│                                               │ viscosity. [1]</s>                           │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ <s>Why didn't bad eyesight get naturally      │ <s>Why didn't bad eyesight get naturally     │ [0.4956, 0.5044] │
│ selected out?                                 │ selected out?                                │                  │
│                                               │                                              │                  │
│ A lot of people have awful eyesight that      │ A lot of people have awful eyesight that     │                  │
│ would make them almost useless in nature, and │ would make them almost useless in nature,    │                  │
│ it's a high enough percentage of the          │ and it's a high enough percentage of the     │                  │
│ population such that I don't think its        │ population such that I don't think its       │                  │
│ survival can be attributed to the same        │ survival can be attributed to the same       │                  │
│ reasons as, say, blindness being present. Did │ reason

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>why does moving an object, that's covered  │ <s>why does moving an object, that's covered │ [0.462, 0.538]   │
│ in a liquid, up make the liquid drip faster?  │ in a liquid, up make the liquid drip faster? │                  │
│                                               │                                              │                  │
│ My Chinese chicken does this when dipped in   │ My Chinese chicken does this when dipped in  │                  │
│ honey sauce. When I move it up to the tray    │ honey sauce. When I move it up to the tray   │                  │
│ from the sauce which is lower it drips really │ from the sauce which is lower it drips       │                  │
│ fast.                                         │ really fast.                                 │                  │
│ When an object covered in a liquid is moved   │ Gas rises because it is less dense than the  │                  │
│ up, it is causing the temperature of the      │ liquids or solids that surround it.          │                  │
│ liquid to drop [1]. When the temperature of a │ Similarly, waves form in a fluid when the    │                  │
│ liquid drops, it causes condensation, which   │ fluid is repeatedly pulled down by gravity,  │                  │
│ is the transfer of heat from the air into the │ causing the speed of wave formation to       │                  │
│ liquid [1,3]. The cooling process is hastened │ exceed the speed at which the waves          │                  │
│ by the formation of water droplets on the     │ themselves grow. As a result, a fluid can    │                  │
│ object, which suck heat from the air and      │ form strands before the waves grow to the    │                  │
│ transfer it directly into the cold liquid     │ length at which they break apart the         │                  │
│ [1].</s>                                      │ strands. [1, 2] This effect can also be seen │                  │
│                                               │ when honey is dripped from a spoon: the      │                  │
│                                               │ viscosity of honey is not a factor in        │                  │
│                                               │ forming strands since gravity pulls the      │                  │
│                                               │ honey into a long string regardless of       │                  │
│                                               │ viscosity. [1]</s>                           │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ <s>Why didn't bad eyesight get naturally      │ <s>Why didn't bad eyesight get naturally     │ [0.5661, 0.4339] │
│ selected out?                                 │ selected out?                                │                  │
│                                               │                                              │                  │
│ A lot of people have awful eyesight that      │ A lot of people have awful eyesight that     │                  │
│ would make them almost useless in nature, and │ would make them almost useless in nature,    │                  │
│ it's a high enough percentage of the          │ and it's a high enough percentage of the     │                  │
│ population such that I don't think its        │ population such that I don't think its       │                  │
│ survival can be attributed to the same        │ survival can be attributed to the same       │                  │
│ reasons as, say, blindness being present. Did │ reason

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>why does moving an object, that's covered  │ <s>why does moving an object, that's covered │ [0.4867, 0.5133] │
│ in a liquid, up make the liquid drip faster?  │ in a liquid, up make the liquid drip faster? │                  │
│                                               │                                              │                  │
│ My Chinese chicken does this when dipped in   │ My Chinese chicken does this when dipped in  │                  │
│ honey sauce. When I move it up to the tray    │ honey sauce. When I move it up to the tray   │                  │
│ from the sauce which is lower it drips really │ from the sauce which is lower it drips       │                  │
│ fast.                                         │ really fast.                                 │                  │
│ When an object covered in a liquid is moved   │ Gas rises because it is less dense than the  │                  │
│ up, it is causing the temperature of the      │ liquids or solids that surround it.          │                  │
│ liquid to drop [1]. When the temperature of a │ Similarly, waves form in a fluid when the    │                  │
│ liquid drops, it causes condensation, which   │ fluid is repeatedly pulled down by gravity,  │                  │
│ is the transfer of heat from the air into the │ causing the speed of wave formation to       │                  │
│ liquid [1,3]. The cooling process is hastened │ exceed the speed at which the waves          │                  │
│ by the formation of water droplets on the     │ themselves grow. As a result, a fluid can    │                  │
│ object, which suck heat from the air and      │ form strands before the waves grow to the    │                  │
│ transfer it directly into the cold liquid     │ length at which they break apart the         │                  │
│ [1].</s>                                      │ strands. [1, 2] This effect can also be seen │                  │
│                                               │ when honey is dripped from a spoon: the      │                  │
│                                               │ viscosity of honey is not a factor in        │                  │
│                                               │ forming strands since gravity pulls the      │                  │
│                                               │ honey into a long string regardless of       │                  │
│                                               │ viscosity. [1]</s>                           │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ <s>Why didn't bad eyesight get naturally      │ <s>Why didn't bad eyesight get naturally     │ [0.5103, 0.4897] │
│ selected out?                                 │ selected out?                                │                  │
│                                               │                                              │                  │
│ A lot of people have awful eyesight that      │ A lot of people have awful eyesight that     │                  │
│ would make them almost useless in nature, and │ would make them almost useless in nature,    │                  │
│ it's a high enough percentage of the          │ and it's a high enough percentage of the     │                  │
│ population such that I don't think its        │ population such that I don't think its       │                  │
│ survival can be attributed to the same        │ survival can be attributed to the same       │                  │
│ reasons as, say, blindness being present. Did │ reason

KeyboardInterrupt: 

: 

## Result 

Initially Promising. Model starts to learn. 

Step	Training Loss	Validation Loss	Accuracy  
1	0.726800	0.693517	0.497424  
2	0.681000	0.693581	0.502576  
3	0.711900	0.693462	0.497424  
4	0.703400	0.693256	0.499714  
5	0.694500	0.693095	0.497424  
6	0.687900	0.692943	0.518031  
7	0.680100	0.692809	0.522038  
8	0.697600	0.692696	0.521465  
9	0.704100	0.692581	0.528334  
10	0.707400	0.692514	0.527189   
11	0.680300	0.692380	0.530052   
12	0.701300	0.692229	0.530624   
13	0.702300	0.692087	0.536920   
14	0.689100	0.691951	0.542072   
15	0.687900	0.691848	0.546651    